In [1]:
from Methods import *

In [2]:
# render figures for the paper

SR = 48000
d = np.array([0, SR // 60])
T = 0.125

fs = [100, 110, 120, 130, 140]
sinusoids = [sinusoid(f, T) for f in fs]
squares = [square(f, T) for f in fs]
saws = [saw(f, T) for f in fs]

harms = 7
sinstacks = [sum(sinusoid(n * f, T) / n ** 2 for n in range(1, harms+1)) for f in fs]
squarestacks = [sum(square(n * f, T) / n ** 2 for n in range(1, harms+1)) for f in fs]
sawstacks = [sum(saw(n * f, T) / n ** 2 for n in range(1, harms+1)) for f in fs]

collections = [sinusoids, squares, saws, sinstacks, squarestacks, sawstacks]

for collection in collections:
    for i in range(len(fs)):
        collection[i] /= np.max(np.abs(collection[i]))

In [ ]:
timbre_vs_frequency(collections, [0,800], '1.basic')
timbre_vs_frequency(collections, [0,191,307,491,797], '2.5d')
timbre_vs_frequency(collections, np.cumsum(range(20)), '3.20d')
timbre_vs_frequency(collections, list(range(96)), '4.highd')

In [4]:
# gets dict of filepaths for orchestral sounds to visualize

# modify to point to your installation of OrchideaSOL
# (the directory given should be the one containing folders "Winds", "Strings", "PluckedStrings", etc.)
# SOLPath = "/media/amc/T7 Shield/OrchideaSOL2020/"
SOLPath = '/Users/amc/Downloads/OrchideaSOL2020/'
file_dict = get_file_dict(SOLPath)

In [5]:
# this cell makes a dictionary of filenames for a variety of instruments, at each dynamic and at a range of 
# pitches. to deal with the varying ranges of the instruments, and to allow for comparison of one pitch played by
# many instruments, we've broken the instrument list into "high" and "low" instruments. 
treble = ['ASax', 'ClBb', 'Fl', 'Va', 'Hp', 'Ob', 'Gtr']
low = ['Acc', 'Bn', 'Hn', 'BTb', 'Tbn', 'Hp', 'Vc', 'Cb']

flattened1 = get_flattened(file_dict, treble)
flattened2 = get_flattened(file_dict, low)

trimmed1 = {(i, p, d) : f for (i,p,d),f in flattened1.items() if ntom(p) in [60, 62, 64, 66, 67, 68]}
trimmed2 = {(i, p, d) : f for (i,p,d),f in flattened2.items() if ntom(p) in [36, 43, 48, 52, 55, 58]}

# master dict of filenames
trimmed = {(i,p,d) : f for (i,p,d), f in [*flattened1.items(), *flattened2.items()]}
print(*trimmed, sep = '\n')

('ASax', 'C5', 'pp')
('ASax', 'C5', 'ff')
('ASax', 'C5', 'mf')
('ASax', 'D4', 'pp')
('ASax', 'D4', 'ff')
('ASax', 'D4', 'mf')
('ASax', 'G4', 'pp')
('ASax', 'G4', 'ff')
('ASax', 'G4', 'mf')
('ASax', 'C#4', 'pp')
('ASax', 'C#4', 'ff')
('ASax', 'C#4', 'mf')
('ASax', 'D5', 'pp')
('ASax', 'D5', 'ff')
('ASax', 'D5', 'mf')
('ASax', 'D#4', 'pp')
('ASax', 'D#4', 'ff')
('ASax', 'D#4', 'mf')
('ASax', 'B3', 'pp')
('ASax', 'B3', 'ff')
('ASax', 'B3', 'mf')
('ASax', 'C#5', 'pp')
('ASax', 'C#5', 'ff')
('ASax', 'C#5', 'mf')
('ASax', 'A5', 'pp')
('ASax', 'A5', 'ff')
('ASax', 'A5', 'mf')
('ASax', 'F#4', 'pp')
('ASax', 'F#4', 'ff')
('ASax', 'F#4', 'mf')
('ASax', 'F4', 'pp')
('ASax', 'F4', 'ff')
('ASax', 'F4', 'mf')
('ASax', 'D#5', 'pp')
('ASax', 'D#5', 'ff')
('ASax', 'D#5', 'mf')
('ASax', 'E5', 'pp')
('ASax', 'E5', 'ff')
('ASax', 'E5', 'mf')
('ASax', 'C4', 'pp')
('ASax', 'C4', 'ff')
('ASax', 'C4', 'mf')
('ASax', 'F5', 'pp')
('ASax', 'F5', 'ff')
('ASax', 'F5', 'mf')
('ASax', 'F#5', 'pp')
('ASax', 'F#5', 'f

In [6]:
# synthesize waveforms

f0 = 110
T = 12

# LFO going from zero to one, then back to zero, over T seconds
lfo = (1 + sinusoid(1 / T, p = 0.25, T = T)) / 2

synthetics = [varying_brightness(f0, lfo, T, 64, 0.5, 0.99), \
              glissando(f0, lfo, T, 7, 0.5, 3), \
              metallic(2 * f0, lfo, T, 9, 0.7, 0.95, 1.05), \
              noisy(f0, lfo, T, 9, 0.7, 1 / SR, 1)]

In [7]:
def render_frames(x, params, ds, basepath, k = 2, pad = 1.1, fps = 60):
    trajectories = [None for d in ds]
    baseses = [None for d in ds]
    distanceses = [None for d in ds]
    rainbows = [None for d in ds]
    boxes = [None for d in ds]
    segmentses = [None for d in ds]
    darknesseses = [None for d in ds]
    colorses = [None for d in ds]

    K = len(x)
    chunks = int(K / params['SR'] * fps)

    for m, delays in enumerate(ds):
        trajectories[m], baseses[m], distanceses[m], _ = analyze(x, np.array(delays), k, alpha = 0.999, delta = 0.05, pad = True, mode = 'svd', normalize = True)

        rainbows[m] = plt.cm.rainbow(np.linspace(0, 1, len(ds[m])))
        boxes[m] = np.max(np.abs(trajectories[m]))

        segmentses[m] = np.zeros((K - 1, 2, k))
        segmentses[m][:,0] = trajectories[m][:-1]
        segmentses[m][:,1] = trajectories[m][1:]

    for m, delays in enumerate(ds):
        darknesseses[m] = 0.9 * (1 - (distanceses[m] / (np.max(distanceses) + 0.2) + (1 - np.max(distanceses) / (np.max(distanceses) + 0.2))))
        darknesseses[m] = np.clip(darknesseses[m], 0, 1)
        colorses[m] = np.zeros((K - 1, 4))

        for j, a in enumerate(colorses[m]):
            colorses[m][j] = (darknesseses[m][j], darknesseses[m][j], darknesseses[m][j], 1)

    try:
        os.mkdir(f'{basepath}/temp')
    except:
        print('temp directory exists already. Removing it.')
        !rm -r {basepath}/temp
        os.mkdir(f'{basepath}/temp')

    for i in (tqdm(range(chunks)) if export else range(chunks)): 
        split = slice((K * i) // chunks, (K * (i + 1)) // chunks)
        fig, ax = plt.subplots(ncols = 2, nrows = 2)

        for p, (trajectory, bases, distances, rainbow, box, segments, darknesses, colors) in enumerate(zip(trajectories, baseses, distanceses, rainbows, boxes, segmentses, darknesseses, colorses)):

            a = ax[p // 2, p % 2]
            a.set_xlim([pad * -box, pad * box])
            a.set_ylim([pad * -box, pad * box])
            a.set_aspect('equal')
            a.set_axis_off()

            lc = LineCollection(segments[split,:], color = colors[split], alpha = 0.75, linewidth = 1, path_effects=[path_effects.Stroke(capstyle="round")])
            a.add_collection(lc)

            for n, c in zip(range(len(delays)), rainbow):
                C = box * bases[split, n].T
                a.plot(*C, c = c, linewidth = 0.75, alpha = 0.5, solid_capstyle='round')

        imgname = f'{basepath}/temp/step' + str(i).zfill(int(np.log10(chunks) + 1)) + '.png'
        plt.savefig(imgname, dpi = 300)
        plt.close('all')


def make_video(x, params, basepath, fps = 60):
    !ffmpeg -y -framerate {fps} -pattern_type glob -i '{basepath}/temp/step*.png' -vcodec libx264 -vf format=yuv420p {basepath}/temp/animated.mp4 -loglevel warning

    if params['synth']:
        # scaled = np.int32(x[:K] * 2 ** 29)
        scipy.io.wavfile.write(f'{basepath}/temp/audio.wav', params['SR'], x[:K]) # if no audio file, synthesize one
        !ffmpeg -y -i Exports/temp/animated.mp4 -i Exports/temp/audio.wav -c:v copy -map 0:v:0 -map 1:a:0 -c:a aac -b:a 192k Exports/temp/video.mp4 -loglevel warning
    else:
        shutil.copyfile(params['path'], f'{basepath}/temp/audio.wav') # otherwise copy audio file for availability

    !ffmpeg -y -i {basepath}/temp/animated.mp4 -i {basepath}/temp/audio.wav -c:v copy -map 0:v:0 -map 1:a:0 -c:a aac -b:a 192k {basepath}/temp/video.mp4 -loglevel warning

    if params['synth']:
        os.rename(f'{basepath}/temp/video.mp4', f'{basepath}/{params["name"]}.{time.strftime("%Y.%m.%d, %H.%M.%S")}.mp4')
    else:
        os.rename(f'{basepath}/temp/video.mp4', f'{basepath}/{instrument}.{pitch}.{dynamic}.{time.strftime("%Y.%m.%d, %H.%M.%S")}.mp4')

In [ ]:
basepath = 'Exports'
ds = [[0,800], [0,191,307,491,797], np.cumsum(range(20)), list(range(96))]

# generate synthetic waveforms
for x, params in synthetics:
    render_frames(x, params, ds, basepath)
    make_video(x, params, basepath)
    
# generate orchestral waveforms
for (instrument, pitch, dynamic), filename in trimmed.items():
    SR, rawX = scipy.io.wavfile.read(filename)
    x = rawX.astype('double')
    x /= np.max(np.abs(x))

    params = {
        'synth': False,
        'file': file,
        'path': filename,
        'SR': SR
    }

    render_frames(x, params, ds, basepath)
    make_video(x, params, basepath)